In [1]:
# ====================================================
# Required Libraries
# ====================================================

import os
import gc
import re
import sys
import time
import math
import random
import warnings
import json
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForPreTraining
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

tokenizers.__version__: 0.21.0
transformers.__version__: 4.49.0
env: TOKENIZERS_PARALLELISM=true
cpu


In [29]:
# ====================================================
# Configuration (Hyper Parameters Value)
# ====================================================

class CFG:
    debug=False # want to debug or not 
    apex=True # for faster training
    print_freq= 300
    num_workers=4 
    model= "csebuetnlp/banglabert" 
    #"csebuetnlp/banglabert_large", "sagorsarker/bangla-bert-base","csebuetnlp/banglabert" 
    # "xlm-roberta-base" , ai4bharat/IndicBERTv2-MLM-only    
    epochs=10
    learning_rate=2e-5 
    eps=1e-6
    betas=(0.9, 0.999) # for adam optimizer
    batch_size= 8 #32  # batch size
    max_len=512
    weight_decay=0.01 # for adam optimizer regulaization parameter
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['binary_label'] #target columns
    seed=42 # seed no. for random initialization 
    train=True
    num_class = None # Number of class in your dataset
    mode = "tiny_based" #"cls_based", "attention_based", "lstm_based"


In [30]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'


train_df = pd.read_csv(train_file, sep="\t")
val_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")




In [31]:
l2id = {
    'None': 0,
    'Religious Hate': 1,
    'Sexism': 2,
    'Political Hate': 3,
    'Profane': 4,
    'Abusive': 5
}
id2l = {v: k for k, v in l2id.items()}


def clean_label(x):
    # handle missing or NaN → "None"
    if pd.isna(x) or x == 'None':
        return 'None'
    # already list-like e.g. ['Abusive']
    if isinstance(x, list):
        return x[0] if len(x) > 0 else 'None'
    # string cases like "[]" or "[Abusive]" or "[Political Hate]"
    x = x.strip("[]").strip()
    if x == "":
        return 'None'
    return x


def process_df(df):
    # Ensure labels are proper lists
    df["label"] = df["label"].apply(clean_label)
    df["label"] = df["label"].fillna("None")
    # Now create binary label
    df["binary_label"] = df["label"].apply(lambda x: 0 if x == "None" else 1)
    df["label_id"] = df["label"].map(l2id)

    

  

    return df

train_df = process_df(train_df)
val_df   = process_df(val_df)

df_train = train_df

In [32]:
CFG.num_class = df_train['label'].nunique()
print(CFG.num_class)

6


In [33]:
df_train

,id,text,label,binary_label,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,0,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,0,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Abusive,1,5
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,0,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Abusive,1,5
...,...,...,...,...,...
35517,790325,তইওয়ানের এত ক্ষমতা হয়নি যে এক টুকরো জায়গা নষ্ট...,None,0,0
35518,328377,চুরের ঘরের চুর হালা,Profane,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Abusive,1,5
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Abusive,1,5


In [34]:
# # If Debugging is True then we will consider a small set of training data

# if CFG.debug:
#     CFG.epochs = 2
#     train = train.sample(frac =.1) 

In [35]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [36]:
# ====================================================
# Logger File
# ====================================================

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(CFG.seed)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Tokenizer, Dataset & Collate Function</h1></span>


In [37]:
# ====================================================
# tokenizer
# ====================================================

PAD_TOKEN = "<pad>"
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.add_special_tokens({"pad_token":PAD_TOKEN})
#tokenizer.padding_side = "right"
tokenizer.max_seq_length = 310
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

In [38]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(df_train['text'].fillna("").values, total=len(df_train))
for text in tk0:
    length = len(tokenizer(text, truncation=True, add_special_tokens=False)['input_ids'])
    lengths.append(length)
    
if max(lengths) + 2 > 512:
    CFG.max_len = 512
else:
    CFG.max_len = max(lengths) + 2 # cls & sep 
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/35522 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
max_len: 512
max_len: 512


In [39]:
# ====================================================
# Dataset Preparation
# ====================================================

def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
#         self.langs = df['lang'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
#         text = f'Lang {self.langs[item]} ' + text
#         self.texts[item] = text
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.long)
        return inputs, label
    

# the colllate function to increase training speed

def collate(inputs): 
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Model Architecture </h1></span>


In [40]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
#             self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
#             self.model = AutoModelForCausalLM.from_pretrained(
#                 self.cfg.model,
#                 device_map="auto",
#                 trust_remote_code = True,
#                 config=self.config
#             )
            self.model = AutoModelForPreTraining.from_pretrained(cfg.model, config=self.config)
 
            self.model.resize_token_embeddings(len(tokenizer),pad_to_multiple_of=8)
        else:
            self.model = AutoModel.from_config(self.config)
        
        # defining attention network for attention scores 
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1))
        
        self.linear = nn.Linear(768, 768*2)
        self.lstm = nn.LSTM(768*2, self.config.hidden_size)
        
        self._init_weights(self.attention)
        self.concat_pool = nn.Linear(self.config.hidden_size*2, self.config.hidden_size)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.num_class)
        self._init_weights(self.fc)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
#         last_hidden_states = outputs.last_hidden_state # word level representation of last hiddent state
        
        if self.cfg.mode == "attention_based":
            # attention based sentence representation
            last_hidden_states = outputs.last_hidden_state
            weights = self.attention(last_hidden_states)
            feature = torch.sum(weights * last_hidden_states, dim=1)
            
            cls_token_feature = last_hidden_states[:, 0, :]
#             print(feature.shape, cls_token_feature.shape)
#             combine_feature = torch.cat([feature, cls_token_feature], dim = -1)
#             feature = self.concat_pool(combine_feature)
            feature += cls_token_feature
            
        if self.cfg.mode == "cls_based":
#             print(f"Available attributes in outputs: {outputs.__dict__.keys()}")
            # [CLS] Token Repr
            last_hidden_states = outputs.last_hidden_state
            feature = last_hidden_states[:, 0, :]
            weights= None
            
        if self.cfg.mode == 'tiny_based':
#             print(outputs.logits)
#             print(f"Available attributes in outputs: {outputs.__dict__.keys()}")
            last_hidden_states = outputs.hidden_states[-1]
            feature = last_hidden_states[:, 0, :]
            weights= None
            
        if self.cfg.mode == "lstm_based":
            last_hidden_states = outputs.last_hidden_state
            x = last_hidden_states[:, 0, :]
            x = self.linear(x)
            feature, _ = self.lstm(x)
            weights= None

        return feature, weights

    def forward(self, inputs):
        feature, weights = self.feature(inputs)
        output = self.fc(feature)
        return output, weights


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Helpler functions for Training  </h1></span>

<font size="3">Few important function are created here.</font>

1. <i>AverageMeter</i> - To compute & store the average
2. <i>asMinutes</i> - To calculate the time
3. <i>timeSince</i> - To compute training & validation time
4. <i>train_fn</i> - Calculation of forward & backward pass for a single epoch in training data
5. <i>valid_fn</i> - Calculation of forward & backward pass for a single epoch in validation data


In [41]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, device):

    
    # Enabling Model Training Mode
    model.train()
     
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex) # using Automatic Mixed Precision (AMP) for speed up
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    
    for step, (inputs, labels) in enumerate(train_loader): # iterate over the training data 
        inputs = collate(inputs) # the collate function I discussed for speeding up training
        
        for k, v in inputs.items():
            inputs[k] = v.to(device)  # formatting the input to feed into the transformer model 
        labels = labels.to(device) 
        batch_size = labels.size(0)
        
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds, _ = model(inputs) 
            loss = criterion(y_preds.view(-1, CFG.num_class), labels.view(-1))

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        scaler.scale(loss).backward() # backpropagation
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm) # clipping the gradient
        
        losses.update(loss.item(), batch_size)
        
        # Updating weights via optimizer & scaler
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm))

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    
    for step, (inputs, labels) in enumerate(valid_loader): # iterate over the validation data 
        inputs = collate(inputs)
        
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad(): # we don't need to store the gradients w.r.t validation data
            y_preds, _ = model(inputs)
            loss = criterion(y_preds.view(-1, CFG.num_class), labels.view(-1))
            
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Training Loop </h1></span>

In [42]:
from sklearn.metrics import f1_score, classification_report

def get_score(y_trues, y_preds):
    y_predicted = y_preds.argmax(axis=1)  # Convert probabilities to class predictions
    macro_f1 = f1_score(y_trues, y_predicted, average='macro')
#     print(classification_report(y_trues, y_predicted, digits=4))
    return macro_f1

In [45]:
# ====================================================
# train loop
# ====================================================
def train_loop():

    # ====================================================
    # loader
    # ====================================================
    
    train_dataset = TrainDataset(CFG, df_train) # training dataset formatting 
    valid_dataset = TrainDataset(CFG, val_df) # validation dataset formatting

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True) # train dataloader
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False) # validation dataloader

    valid_labels = val_df[CFG.target_cols].values
    
    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)  # initializing the model
    torch.save(model.config, OUTPUT_DIR+'config.pth') # saving the model configuration 
    model.to(device) # GPU Config
    
    optimizer = AdamW(model.parameters(), lr=CFG.learning_rate, eps=CFG.eps, betas=CFG.betas) # declaring the optimizer
    
    criterion = nn.CrossEntropyLoss(reduction="mean")
    best_score = 0


    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train function 
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

        # eval function 
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        
        if best_score < score: # Saving the best model w.r.t the score 
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_score{best_score:.4f}_best.pth")

#     predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_score{best_score:.4f}_best.pth", 
#                              map_location=torch.device('cpu'))['predictions']
#     final_pred = predictions.argmax(axis=1)
#     final_pred = final_pred.tolist()
#     df_dev[f"pred_label"] = final_pred

    torch.cuda.empty_cache()
    gc.collect()
    
    return best_score

In [ ]:
# ====================================================
# the training
# ====================================================
    
if __name__ == '__main__':
    
    if CFG.train:
        best_score = train_loop()

ElectraConfig {
  "_name_or_path": "csebuetnlp/banglabert",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

ElectraConfig {
  "_name_or_path": "csebuetnlp/banglabert",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_dropout": 0.0,
 

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Epoch: [1][0/4440] Elapsed 0m 5s (remain 393m 4s) Loss: 2.0055(2.0055) Grad: 9.6964  
Epoch: [1][300/4440] Elapsed 14m 55s (remain 205m 17s) Loss: 0.5317(0.6306) Grad: 7.3737  


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center">  Inference </h1></span>


<font color='#3498DB'> <h3> <a id ="section11a"> <b> Configuration for Inference</b> </a> </h3> </font>

<font size="3"> The basic and important configuration for infernce is described here along with some function & other stuff.</font>


In [ ]:
# ====================================================
# CFG for testing
# ====================================================

class CFG_Test:
    num_workers=4
    path="./"
    config_path=path+'config.pth'
    model=CFG.model
    batch_size=CFG.batch_size
    target_cols=CFG.target_cols
    seed=CFG.seed
    num_class = CFG.num_class
    mode = CFG.mode
    
CFG_Test.tokenizer = AutoTokenizer.from_pretrained(CFG_Test.path+'tokenizer/') # load the saved pretrained tokenizer

In [ ]:
def get_logger(filename='inference'): # infernece logger file
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

<font color='#3498DB'> <h3> <b> Model Loading for Inference</b> </h3> </font>

<font size="3"> Dataset for predicting on the test data and Model Loading for inference are done in this section </font>

In [ ]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True)
    
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['Text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

<font color='#3498DB'> <h3> <b> Prediction on Test Data</b></h3> </font>

<font size="3"> An inference function is made for predicting on the test data. Then finally, loading the previously saved model for each fold and taking prediction on test dataset for each fold. Then, take the average of the each of prediction is considered as model final prediction, </font>

In [ ]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0: # iterate over the test data
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds, _ = model(inputs) # considering the logits only
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from transformers import DataCollatorWithPadding
test_dataset = TestDataset(CFG_Test, df_test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_Test.batch_size,
                         shuffle=False,
                         collate_fn=DataCollatorWithPadding(tokenizer=CFG_Test.tokenizer, padding='longest'))
                         

model = CustomModel(CFG_Test, config_path=CFG_Test.config_path, pretrained=True)
state = torch.load(CFG_Test.path+f"{CFG_Test.model.replace('/', '-')}_score{best_score:.4f}_best.pth",
                   map_location=torch.device('cpu')) # loading the saved model

model.load_state_dict(state['model'])
prediction = inference_fn(test_loader, model, device)
del model, state; gc.collect()
torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

print('\nThe Classification Report is as follows\n')
final_prediction = prediction.argmax(axis = 1)
print(classification_report(df_test['Label'].tolist(), final_prediction, digits = 4))

In [ ]:
df_test['Pred_label'] = final_prediction
df_test.to_csv('cuetbanglabert-hate-test.csv', index=False)

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# from transformers import DataCollatorWithPadding
# test_dataset = TestDataset(CFG_Test, df_test)
# test_loader = DataLoader(test_dataset,
#                          batch_size=CFG_Test.batch_size,
#                          shuffle=False,
#                          collate_fn=DataCollatorWithPadding(tokenizer=CFG_Test.tokenizer, padding='longest'))
                         

# model = CustomModel(CFG_Test, config_path=CFG_Test.config_path, pretrained=True)
# state = torch.load(CFG_Test.path+f"{CFG_Test.model.replace('/', '-')}_score{best_score:.4f}_best.pth",
#                    map_location=torch.device('cpu')) # loading the saved model

# model.load_state_dict(state['model'])
# prediction = inference_fn(test_loader, model, device)
# del model, state; gc.collect()
# torch.cuda.empty_cache()
    


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Evaluation on Test Dataset </h1></span>


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#C01F4D; border-radius: 100px 100px; text-align:center"> Thanks for Reading </h1></span>